## The Walks of Nathan Lowell
A look at Nate's path to the back gate.

The journey begins [#tommw](https://www.nathanlowell.com/tommw/)

Archive File Structure

``` python
{'meta': {'ids':[
                '12345',
                '67890',
                ...]
          'last_updated': 'UTC Timestamp',
         }
 'data': [
     {'link':URl string,
      'condition': string,
      ...}
         ]
}
 

```


In [1]:
from io import BytesIO
import json
from PIL import Image
import shutil

import arrow
from bs4 import BeautifulSoup
import requests

import photo as pto

[#tommw](https://www.nathanlowell.com/tommw/) is a cross channel social media campaign architeched and executed by science fiction author Nathan Lowell. 

There are 4 planks to Nathan's campain:
1. A blog hosted at https://www.nathanlowell.com/tommw/
2. A photo stream hosted on [FlickR](https://www.flickr.com/photos/nlowell/)
3. Podcast feed via [RSS](http://www.nathanlowell.com/tommw/feed/)
4. Posts to twitter with the #tommw hastag on [@nlowell on twitter](https://twitter.com/nlowell)

In [2]:
# Functions to manage the archive file

def get_archive():

    # Load the archive file
    try:
        with open('archive.json', 'r') as file:
            archive= json.loads(file.read())
        print('Archive loaded.')
        
    except:
        # Create archive data structure.

        archive = {'meta':{'ids':[],
                           'audio':[],
                          'last_updated':'New File',
                          },
                  'data':[],
                  }
        print('No archived file, building new archive.json')
        
    return archive

def save_archive(archive):
    
    with open('archive.json', 'w') as file:
        archive['meta']['last_updated'] = str(arrow.now())
        file.write(json.dumps(archive, indent=4))
        
    return True

def backfill_dates(archive):
    '''Helper function to convert timestamps
    to a date. Date is used to match audio files to photos.'''

    for walk in archive['data']:

        timestamp = walk['timestamp']

        date = timestamp.split(' ')[0]

        walk['date'] = date

    return archive

    

In [3]:
# Future podcast.py

import requests
import tinytag


def get_mp3(podcast_link, archive):

    mp3_name = podcast_link.split('/')[-1]

    mp3_file = 'audio/' + mp3_name

    if mp3_name in archive['meta']['audio']:
        # skip files already downloaded
        return mp3_file, archive

    file = requests.get(podcast_link)

    if file.status_code != 200:

        return False

    with open(mp3_file, 'wb') as f:
        f.write(file.content)

    archive['meta']['audio'].append(mp3_name)

    return mp3_file, archive



def get_id3_tag(mp3_file):

    id_info = tinytag.TinyTag.get(mp3_file).as_dict()

    title = id_info.get('title')

    if title:
        date = title.split(' ')[-1]
    else:
        date = 'unknown'

    id_info['date'] = date

    id_info['audio'] = mp3_file
    

    return id_info


def match_photo_date(id_info, archive):
    '''Iterates through the archive to match audio
    to a photo record'''

    for walk in archive['data']:

        if walk['date'] == id_info['date']:

            walk.update(id_info)
            mp3_file = id_info.get('audio')
            
            if mp3_file.find('.mp3') > 1:
                mp3_name = mp3_file.strip('audio/')
                archive['meta']['audio'].append(mp3_name)
                
            else:
                print('Unable to get mp3 info from', id_info)

    return archive


def build_audio_archive(podcast_link, archive):
    # TODO complete this function
    # Parse URL into base, date, file_name
    podcast_link.rsplit('/', maxsplit=2)

    pass


In [4]:
# Future Main

# Path to temp directory
latest = 'latest/'

# Import the archive from disk
archive = get_archive()

# Scrape the blog for links to the other media components
blog_page = requests.get("https://www.nathanlowell.com/tommw/")
soup = BeautifulSoup(blog_page.text, 'html.parser')

# Find a flickr link
flickr = soup.find_all(href=True, attrs={'data-flickr-embed':'true'})

# Find a podcast link
podcast_link = soup.find(class_="powerpress_link_pinw").get('href')

if not flickr:
    print('No flickr links found on blog home page.')
    flicker_link = None
    
else:
    flickr_link = flickr[0]['href']

    
if len(archive['meta']['ids']) < 400:
    archive = pto.build_flickr_archive(flickr_link, archive)
    

# Process the photostream
stream_url = pto.get_stream_url(flickr_link)
stream = pto.get_photostream(stream_url)
archive = pto.process_photo_stream_page(stream, archive)

# Check the album page
album_url = 'https://www.flickr.com/photos/nlowell/albums/72157626736309035'
stream = pto.get_photostream(album_url, album=True)
archive = pto.process_photo_stream_page(stream, archive)


# Save the audio file
mp3_file, archive = get_mp3(podcast_link, archive)

# Save the data to archive.json
save_archive(archive)
    

Archive loaded.
no #tommow tag found


True

In [5]:
archive = pto.build_flickr_archive(album_url, archive, album=True)

https://www.flickr.com/photos/nlowell/albums/72157626736309035/page1
https://www.flickr.com/photos/nlowell/albums/72157626736309035/page2
https://www.flickr.com/photos/nlowell/albums/72157626736309035/page3
no #tommow tag found
https://www.flickr.com/photos/nlowell/albums/72157626736309035/page4
no #tommow tag found
no #tommow tag found
no #tommow tag found
https://www.flickr.com/photos/nlowell/albums/72157626736309035/page5
https://www.flickr.com/photos/nlowell/albums/72157626736309035/page6


In [10]:
save_archive(archive)

True

In [ ]:
# Quick check for blocks of missing photos

times = []
for photo in archive['data']:
    times.append(photo['date'])

times.sort()
times
    

In [ ]:
mp3_file = 'audio/tommw-2018-12-21.mp3'
latest = 'latest/'

In [ ]:
mp3_file.find('.mp3')

In [ ]:
import scipy.io.wavfile as wv
import pydub
import tinytag


In [ ]:
mp3_audio = pydub.AudioSegment.from_mp3(mp3_file)

In [ ]:
#convert to wav
mp3_audio.export(latest+"unprocessed.wav", format="wav")

#read wav file
rate,audData=wv.read(latest+"unprocessed.wav")

# Calculate track time
track_time = (audData.shape[0] / rate)/60



In [ ]:
for walk in archive['data']:
    print(walk['date'])

In [ ]:
archive['data'][0]

In [ ]:
archive = backfill_dates(archive)

In [ ]:
# Future Twitter.py
import toml
import twython as tw

cred = toml.load('credentials.toml')

mm = {'Jan':'01',
     'Feb':'02',
     'Mar':'03',
     'Apr':'04',
     'May':'05',
     'Jun':'06',
     'Jul':'07',
     'Aug':'08',
     'Sep':'09',
     'Oct':'10',
     'Nov':'11',
     'Dec':'12'}

def get_new_twitter_token(cred):
    tw_creds = cred['api']['twitter']

    twitter = tw.Twython(tw_creds['consumer_key'],
                              tw_creds['consumer_secret'],
                              oauth_version=2)

    ACCESS_TOKEN = twitter.obtain_access_token()
    
    cred['api']['twitter']['access_token'] = ACCESS_TOKEN
    
    with open('credentials.toml', 'w') as f:

        toml.dump(cred, f)

    return ACCESS_TOKEN

def get_token(cred):

    token = cred['api']['twitter'].get('access_token')


    if token == None:
        token = get_new_twitter_token(cred)
        
    return token

def get_twitter(token):
    
    twitter = tw.Twython(tw_creds['consumer_key'],
                         access_token=token)

def format_tweet_time(tweet_time):
    '''Transforms tweet date time
    format "5:11 AM - 16 Jun 2011"
    to timestamp "2011-06-16 05:11:00" '''

    dt_parts = tweet_time.split(' ')
    year = dt_parts[-1]
    
    # Convert text month to digits
    month = mm[dt_parts[-2]]
    
    day = dt_parts[-3]
    raw_time = dt_parts[0].split(':')
    hour = raw_time[0].rjust(2,'0')
    minute = raw_time[-1]
    second = '00'

    # Adjust time for am vs pm
    if dt_parts[1]=='PM':
        hour_num = int(hour) + 12
        hour = str(hour)


    tw_date = '-'.join([year,month,day])
    tw_time = ':'.join([hour,minute,second])
    tw_timestamp = ' '.join([tw_date, tw_time])

    return tw_timestamp, tw_date, tw_time

def process_tweet_body(body):

    info = {}
    
    raw_title = body.split('"')[1]

    comma = raw_title.find(',')

    # Find a period with a space after it.
    period = raw_title.find('. ')

    if comma > 0:
        tag_temp = raw_title[:comma]
        wind_link = raw_title[comma:]

    elif comma < 0 and period > 0:

        tag_temp = raw_title[:period]
        wind_link = raw_title[period:]

    else:
        print('Unable to parse title')
        info['tag'] = 'Error'
        return info

    # TODO refactor using deque to improve performance
    
    t_parts = tag_temp.split(' ')
    info['tag'] = t_parts.pop(0)
    info['temp'] = t_parts.pop(0)
    info['sun'] = ' '.join(t_parts)

    w_parts = wind_link.split(' ')
    info['photo_link'] = w_parts.pop()
    info['wind'] = ' '.join(w_parts)
    
    return info

def process_tweet(tweet_url):
    
    tweet = requests.get(tweet_url)
    tsoup = BeautifulSoup(tweet.text, 'html.parser')
    
    body = tsoup.title.text
    info = process_tweet_body(body)
    
    tweet_time = str(tsoup.find(class_="client-and-actions").text.strip())
    tw_time = format_tweet_time(tweet_time)
    
    info['tweet_timestamp'], info['date'], info['time'] = tw_time
    
    info['tweet_id'] = tweet_url.split('/')[-1]
    
    return info

if __name__ == "__main__":
    
    token = get_token(cred)
        


In [ ]:
search_url = 'https://twitter.com/search?l=en&q=%23tommw%20from%3Anlowell%20since%3A2011-06-01%20until%3A2011-07-31&src=typd'

In [ ]:
tweet_url = 'https://twitter.com/nlowell/status/81333156376096768'
tw_info = process_tweet(tweet_url)
tw_info

In [ ]:
archive = get_archive()
archive['data'][20]

In [ ]:
archive['meta']


